In [12]:
import sys
sys.path.append('/Users/marjoriesayer/anaconda/envs/mids_hw1/lib/python2.7/site-packages')

In [13]:
%matplotlib inline

# General Libraries
import re
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss

Load the Crime data.

In [14]:
crime_data = []
crime_labels = []
with open("../../train.csv", "rb") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    first_row = True
    for row in reader:
        data = [y for x, y in enumerate(row) if x != 1]
        name = [y for x, y in enumerate(row) if x == 1]
        
        if first_row:
            first_row = False
            feature_names = data
            print feature_names
        else:
            crime_data.append(data)
            crime_labels.append(name)

['Dates', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']


In [15]:
# Divide the Crime training set into train data, test data, dev data
train_data, train_labels = crime_data[:500000], crime_labels[:500000]
crime_test, crime_test_labels = crime_data[500000:], crime_labels[500000:]
num_test = len(crime_test)
print num_test
dev_data, dev_labels = crime_test[:num_test/2], crime_test_labels[:num_test/2]
test_data, test_labels = crime_test[num_test/2:], crime_test_labels[num_test/2:]
mini_train_data, mini_train_labels = crime_data[200000:300000], crime_labels[200000:300000]

print "Size of train data: ", len(train_data)
print "Size of dev data: ", len(dev_data)
print "Size of test data: ", len(test_data)

378049
Size of train data:  500000
Size of dev data:  189024
Size of test data:  189025


In [16]:
from datetime import datetime
def convert_date(date_val):
    date_list = []
    date, time = date_val.split()
    date_list.append(int(date.split('-')[0]))
    date_list.append(int(date.split('-')[1]))
    date_list.append(int(date.split('-')[2]))
    time_in_min = int(time.split(':')[0]) * 60.0 + int(time.split(':')[1])
    date_list.append(time_in_min)
    date_list.append(int(time.split(':')[0]))
    time_hour = int(time.split(':')[0])
    if time_hour < 6:
        time_of_day = 'Twilight'
    elif time_hour < 12:
        time_of_day = 'Morning'
    elif time_hour < 18:
        time_of_day = 'Afternoon'
    else:
        time_of_day = 'Night'
    date_list.append(time_of_day)
    return date_list

print train_data[0][0]
print convert_date(train_data[0][0])

def find_mean_std(train_data, index):
    data = []
    for row in train_data:
        data.append(float(row[index]))
    
    data_arr = np.array(data, dtype=np.float32)
    print np.mean(data_arr)
    data_dict = {}
    data_dict['mean'] = np.mean(data_arr)
    data_dict['std'] = np.std(data_arr)
    return data_dict

2015-05-13 23:53:00
[2015, 5, 13, 1433.0, 23, 'Night']


In [17]:
districts = ['NORTHERN', 'PARK', 'INGLESIDE', 'BAYVIEW', 'RICHMOND', 'CENTRAL', 'TARAVAL', 'TENDERLOIN', 'MISSION', 'SOUTHERN']
day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daytime = ['Twilight', 'Morning', 'Afternoon', 'Night']

Feature Engineering - Section I:

1) Crime Date - Normalization

2) Longitude & Latitude - Normalization

In [18]:
''' Crime Date - Normalization '''

year_data = []
mon_data = []
day_data = []
time_data = []
time_of_day_data = []
for row in crime_data:
    date_arr = convert_date(row[0])
    year_data.append(float(date_arr[0]))
    mon_data.append(float(date_arr[1]))
    day_data.append(float(date_arr[2]))
    time_data.append(float(date_arr[3]))
    time_of_day_data.append(date_arr[5])
                     

year_arr = np.array(year_data, dtype=np.float32)
mon_arr = np.array(mon_data, dtype=np.float32)
day_arr = np.array(day_data, dtype=np.float32)
time_arr = np.array(time_data, dtype=np.float32)
time_of_day_arr = np.array(time_of_day_data)

date_dict = {}
date_dict['mean_year'] = np.mean(year_arr)
date_dict['std_year'] = np.std(year_arr)                               
date_dict['mean_mon'] = np.mean(mon_arr)
date_dict['std_mon'] = np.std(mon_arr)
date_dict['mean_day'] = np.mean(day_arr)
date_dict['std_day'] = np.std(day_arr)
date_dict['mean_time'] = np.mean(time_arr)
date_dict['std_time'] = np.std(time_arr)


''' Longitude & Latitude - Normalization '''
long_dict = find_mean_std(train_data, 6)
lat_dict = find_mean_std(train_data, 7)

-122.423
37.7675


Preparing the Data

In [19]:

def get_formatted_data(train_data):
    format_data = []
    count = 0
    
    for row in train_data:
        count += 1
        data = []
        
        ''' Data Preparation - Crime date '''
        
        #date_list = convert_date(row[0])
        date_arr = convert_date(row[0])
        
        data.extend(date_arr[:3]) # Normalized date
        
        # Time of the day preparation
        time_of_day = date_arr[5]
        day_time = [1 if time_of_day == t else 0 for t in daytime]
        data.extend(day_time)
        
        #year_val = (float(date_list[0]) - date_dict['mean_year'])/date_dict['std_year']
        #data.append(year_val)
        #mon_val = (float(date_list[1]) - date_dict['mean_mon'])/date_dict['std_mon']
        #data.append(mon_val)
        #day_val = (float(date_list[2]) - date_dict['mean_day'])/date_dict['std_day']
        #data.append(day_val)
        #time_val = (float(date_list[3]) - date_dict['mean_time'])/date_dict['std_time']
        #data.append(time_val)
        
        ''' Data Preparation - Day of the Week '''
        week_day = row[2]
        crime_day = [1 if week_day == d else 0 for d in day] 
        data.extend(crime_day) # Normalized
        
        ''' Data Preparation - District '''
        pdd = row[3]
        pddistrict = [1 if pdd == d else 0 for d in districts] # Normalized
        data.extend(pddistrict)
        
        ''' Data Preparation - Longitude & Latitude '''
        longitude = float(row[6])
        long_norm = (abs(longitude) - abs(long_dict['mean']))/long_dict['std'] # Normalized
        latitude = float(row[7])
        lat_norm = (abs(latitude) - abs(lat_dict['mean']))/lat_dict['std'] # Normalized
        data.append(long_norm)
        data.append(lat_norm)
        
        ''' Data Preparation - Address '''
        
        address = row[5].lower()
        addr = [1 if 'block' in address else 0] # Requires major improvement
        data.extend(addr)
        
        # Explore external map source
        
        ''' Quantify the data '''
        format_data.append(np.array(data, dtype=np.float32))           
        
    return format_data
    
    
crimeX = np.array(get_formatted_data(crime_data), dtype=np.float32)
trainX = np.array(get_formatted_data(train_data), dtype=np.float32)
testX = np.array(get_formatted_data(test_data), dtype=np.float32)
devX = np.array(get_formatted_data(dev_data), dtype=np.float32)
mini_trainX = np.array(get_formatted_data(mini_train_data), dtype=np.float32)
print trainX[:1]

[[  2.01500000e+03   5.00000000e+00   1.30000000e+01   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    1.29296839e-01   2.97121316e-01   0.00000000e+00]]


In [25]:
print trainX[2]

[  2.01500000e+03   5.00000000e+00   1.30000000e+01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   0.00000000e+00   1.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   6.88347891e-02   1.37332428e+00   0.00000000e+00]


In [9]:
print "CrimeX Data shape: ", crimeX.shape
print "CrimeX Label shape: ", np.array(crime_labels).shape
print "testX Data shape: ", testX.shape
print "testX Label shape:", np.array(test_labels).shape
print "Mini TrainX Data shape: ", mini_trainX.shape
print "Mini TrainX Label shape: ", np.array(mini_train_labels).shape

CrimeX Data shape:  (878049, 27)
CrimeX Label shape:  (878049, 1)
testX Data shape:  (189025, 27)
testX Label shape: (189025, 1)
Mini TrainX Data shape:  (100000, 27)
Mini TrainX Label shape:  (100000, 1)


In [21]:
print len(crime_labels[0])

1


In [22]:
train_labels = np.array(train_labels).ravel()
test_labels = np.array(test_labels).ravel()
crime_labels = np.array(crime_labels).ravel()

In [23]:
lr = LogisticRegression(C=0.1, tol = 0.01)
lr.fit(crimeX, crime_labels)
print "Completed the training"

Completed the training


In [24]:
mini_testX, mini_test_labels = testX[:10000], test_labels[:10000]
f1_score = metrics.f1_score(lr.predict(mini_testX), mini_test_labels)
print f1_score

0.324677500419


/Users/marjoriesayer/anaconda/envs/mids_hw1/lib/python2.7/site-packages/sklearn/metrics/classification.py:676: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/Users/marjoriesayer/anaconda/envs/mids_hw1/lib/python2.7/site-packages/sklearn/metrics/classification.py:960: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


f1 score when no regularization: 0.306096502629
with regularization of 0.1: 0.320155666949


In [1]:
print f1_score

NameError: name 'f1_score' is not defined

#Using Jaime Data Here

In [29]:
from feature_engineering import FeatureEngineering
fe = FeatureEngineering()
fe.load_train_data()

Initializing
Loading train data
Shuffle the data and divide it into train, mini_train, dev and test sets
 + Status report:
    - Train data and labels: (580000, 48) - (580000,)
    - Mini train data and labels: (10000, 48) - (10000,)
    - Dev data and labels: (150000, 48) - (150000,)
    - Test data and labels: (148049, 48) - (148049,)


In [31]:
print fe.train_data.shape
print fe.train_labels.shape
print fe.mini_train_data.shape
print fe.mini_train_labels.shape
print fe.dev_data.shape
print fe.dev_labels.shape
print fe.test_data.shape
print fe.test_labels.shape
#print fe.submission_data.shape 

(580000, 48)
(580000,)
(10000, 48)
(10000,)
(150000, 48)
(150000,)
(148049, 48)
(148049,)


In [32]:
lr = LogisticRegression(C=0.1, tol = 0.01)
lr.fit(fe.train_data, fe.train_labels)
print "training completed"

training completed


In [33]:
mini_testX, mini_test_labels = fe.test_data[:10000], fe.test_labels[:10000]
f1_score = metrics.f1_score(lr.predict(mini_testX), mini_test_labels)
print f1_score

0.3172562035


/Users/marjoriesayer/anaconda/envs/mids_hw1/lib/python2.7/site-packages/sklearn/metrics/classification.py:676: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


In [23]:
sub_test_data = []
with open("../../test.csv", "rb") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    first_row = True
    for row in reader:
        data = [y for x, y in enumerate(row)]
        if first_row:
            first_row = False
        else:
            sub_test_data.append(data)
            

def get_formatted_test_data(test_data):
    format_data = []
    count = 0
    
    for row in test_data:
        count += 1
        data = []
        
        date_arr = convert_date(row[1])
        data.extend(date_arr[:3]) # Normalized date
        time_of_day = date_arr[5]
        day_time = [1 if time_of_day == t else 0 for t in daytime]
        data.extend(day_time)
        
        #year_val = (float(date_arr[0]) - date_dict['mean_year'])/date_dict['std_year']
        #data.append(year_val)
        #mon_val = (float(date_arr[1]) - date_dict['mean_mon'])/date_dict['std_mon']
        #data.append(mon_val)
        #mon_val = (float(date_arr[2]) - date_dict['mean_day'])/date_dict['std_day']
        #data.append(day_val)
        #time_val = (float(date_arr[3]) - date_dict['mean_time'])/date_dict['std_time']
        #data.append(time_val)
        
        week_day = row[2]
        crime_day = [1 if week_day == d else 0 for d in day] 
        data.extend(crime_day)
        
        pdd = row[3]
        pddistrict = [1 if pdd == d else 0 for d in districts]
        data.extend(pddistrict)
        
        longitude = float(row[5])
        long_norm = (abs(longitude) - abs(long_dict['mean']))/long_dict['std']
        latitude = float(row[6])
        lat_norm = (abs(latitude) - abs(lat_dict['mean']))/lat_dict['std']
        data.append(long_norm)
        data.append(lat_norm)
        
        if 'block' in row[4].lower():
            address = 1
        else:
            address = 0
        data.append(address)
        format_data.append(np.array(data, dtype=np.float32))           
        
    return format_data
    
    

testX = np.array(get_formatted_test_data(sub_test_data), dtype=np.float32)

        

In [17]:
print "hello"

hello


In [25]:
probs = lr.predict_proba(testX)

In [26]:
print probs.shape
print probs[:1]

(884262, 39)
[[  5.08778624e-03   1.10662398e-01   9.17294038e-05   6.67460252e-04
    4.91912604e-02   1.85602845e-03   1.91787501e-03   3.90408942e-02
    4.46866918e-03   4.70025175e-04   1.49900134e-04   6.37088109e-04
    3.42208852e-03   6.76801238e-03   1.73835233e-04   4.58723724e-03
    1.57442886e-01   1.40903942e-03   2.30785097e-04   5.65454880e-02
    7.59824096e-02   1.13616602e-01   1.59773416e-05   7.68972530e-04
    4.28024260e-03   2.55923180e-02   3.81613786e-03   2.63796327e-02
    4.98827057e-03   2.34718363e-04   3.72114962e-03   4.43382191e-04
    3.99266692e-02   6.08285202e-06   6.19009855e-03   8.00325729e-02
    1.06607402e-01   4.35350999e-02   1.90417739e-02]]


In [27]:
import gzip
with gzip.open('submission-matrix_m1.csv.gz', 'wb') as f:
    out = csv.writer(f, lineterminator='\n')
    out.writerow(['Id'] + list(np.unique(crime_labels)))
    
    for i, prob in enumerate(probs):
        out.writerow([i] + list(prob))
print "Job completed"     

Job completed


In [38]:
clf = RandomForestClassifier(n_estimators=39)
clf.fit(fe.train_data, fe.train_labels)
print "Random Forest Training Completed"


Random Forest Training Completed


In [48]:
print len(clf.feature_importances_)

48


In [49]:
print clf.feature_importances_
weights = {}
for i in range(len(clf.feature_importances_)):
    weights[i] = clf.feature_importances_[i]


[ 0.08734227  0.08964935  0.10317346  0.06769599  0.10951543  0.00369097
  0.00751188  0.00930648  0.00561454  0.01714434  0.01738971  0.01770797
  0.0175217   0.018228    0.01745161  0.01641938  0.00114133  0.00082665
  0.00099346  0.00135296  0.00069472  0.00085993  0.00076635  0.00254565
  0.00110713  0.00104237  0.03606926  0.03953369  0.01632297  0.01901863
  0.0159743   0.01589142  0.0147423   0.01565541  0.01515426  0.01492152
  0.01496826  0.01471213  0.01477213  0.015246    0.01469514  0.01452583
  0.01453494  0.01500887  0.01510898  0.01477813  0.01554223  0.01612998]


In [43]:
mini_testX, mini_test_labels = fe.test_data[:10000], fe.test_labels[:10000]
f1_score = metrics.f1_score(clf.predict(mini_testX), mini_test_labels)
print f1_score
f2 = metrics.f1_score(mini_test_labels, clf.predict(mini_testX), average = 'weighted')

0.307217278116


/Users/marjoriesayer/anaconda/envs/mids_hw1/lib/python2.7/site-packages/sklearn/metrics/classification.py:676: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


In [44]:
print f2

0.272982721884


In [ ]:
# Result with n_estimators = 10 is f1_score = 0.285332771193
# Result with n_estimators = 25 is f1_score = 0.297548971568
# Result with n_estimators = 39 if f1_score = 0.307217278116

In [42]:
print fe.train_labels[:2]

[ 20.  21.]


In [51]:
clf = RandomForestClassifier(n_estimators=39, class_weight = weights)
clf.fit(fe.train_data, fe.train_labels)
print "Random Forest Training Completed"

IndexError: index 39 is out of bounds for axis 0 with size 39

In [31]:
clf_probs = clf.predict_proba(testX)
print clf_probs.shape
print clf_probs[:1]

(884262, 39)
[[ 0.01709402  0.18437118  0.          0.          0.01648352  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.28205128  0.          0.          0.05128205
   0.10683761  0.07692308  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.02564103  0.08547009
   0.          0.          0.05128205  0.05128205  0.02564103  0.02564103]]


In [ ]:
import gzip
with gzip.open('submission-matrix.csv.gz', 'wb') as f:
    out = csv.writer(f, lineterminator='\n')
    out.writerow(['Id'] + list(np.unique(crime_labels)))
    
    for i, prob in enumerate(clf_probs):
        out.writerow([i] + list(prob))
print "Job completed" 